# BUFR Message Decoder for Sub-hourly Data

This notebook demonstrates how to decode BUFR message containing sub-hourly meteorological data using the `eccodes` library.




## Explore a BUFR message with comand bufr_ls

### bufr_ls -p masterTablesVersionNumber sub-hourly.bufr 

>  masterTablesVersionNumber   
     33                                               
       
### bufr_ls -p center,localTablesVersionNumber sub-hourly.bufr
> center localTablesVersionNumber   
     ecmwf                        0

## Explore a BUFR message with comand bufr_dump

### bufr_dump -p  sub-hourly.bufr 

> DataCategory=0 <br>
internationalDataSubCategory=7 <br>
dataSubCategory=178 <br>
masterTablesVersionNumber=33 <br>
localTablesVersionNumber=0 <br>
typicalYear=2024 <br>
typicalMonth=11 <br>
typicalDay=19 <br>
typicalHour=18 <br>
typicalMinute=40 <br>
typicalSecond=0 <br>
numberOfSubsets=1 <br>
observedData=1 <br>
compressedData=0 <br>
unexpandedDescriptors={
      307092, 002001, 005001, 006001, 007001 }





## Setup enviroment
First, we'll install and import the required libraries.

In [11]:
from eccodes import *
import numpy as np
import pandas as pd

## Welcome to BUFR world
These functions will help us process the BUFR messages.

In [12]:
def read_bufr_message(file_path):
    """
    Read a BUFR message from a file.
    
    Args:
        file_path (str): Path to the BUFR file
                
    Returns:
        eccodes.BufrMessage: BUFR message handle
    """
    with open(file_path, 'rb') as f:
        bufr = codes_bufr_new_from_file(f)
        if bufr is None:
            raise ValueError(f"Failed to load BUFR message from {file_path}")
        return bufr

def decode_bufr_message(bufr):
    """
    Decode a BUFR message and extract key metadata and observations.
    
    Args:
        bufr: BUFR message handle
        
    Returns:
        dict: Dictionary containing decoded data
    """
    # Decode the message
    codes_set(bufr, 'unpack', 1)
      
    
    # Extract  meteorological parameters
    try:
        data = {
            'masterTablesVersionNumber': codes_get(bufr, 'masterTablesVersionNumber'),
            'dataCategory': codes_get(bufr, 'dataCategory'),
            'typicalDate': codes_get(bufr, 'typicalDate'),
            'typicalTime': codes_get(bufr, 'typicalTime'),
            'latitude': codes_get(bufr, 'latitude'),
            'longitude': codes_get(bufr, 'longitude'),
            'temperature': codes_get(bufr, 'airTemperature'),
            'temperature significance': codes_get(bufr, 'airTemperature->associatedField'),
            'wind speed': codes_get(bufr, 'windSpeed'),
            'wind speed significance': codes_get(bufr, 'windSpeed->associatedField'),
            'wind direction': codes_get(bufr, 'windDirection'),
            'wind direction significance': codes_get(bufr, 'windDirection->associatedField'),
            'wind direction associated field significance': codes_get(bufr, 'windDirection->associatedField->associatedFieldSignificance'),
        }
    except Exception as e:
        print(f"Warning: Not all parameters could be extracted: {str(e)}")
          
  
    return {'data': data}



## Process BUFR File
Now we can process a BUFR file and analyze its contents.

In [13]:
# Specify your BUFR file path
bufr_file="./data/sub-hourly.bufr"

# Read and decode the BUFR message
bufr = read_bufr_message(bufr_file)

# Decode the BUFR message
decoded_data = decode_bufr_message(bufr)


# Convert decoded BUFR data into a pandas DataFrame
df = pd.DataFrame(decoded_data)

display(df)

# Clean up
codes_release(bufr)

,data
masterTablesVersionNumber,33
dataCategory,0
typicalDate,20241119
typicalTime,184000
latitude,57.36
longitude,-7.38
temperature,275.34
temperature significance,0.0
wind speed,2.9
wind speed significance,0.0


## Associated field and associated field significance
### Let's undersand meaning of these attributes to the BUFR meteorological variables.
 We look first at the BUFR template definition. We can find this information in unexpandedDescriptors. In our case it is 3 07 092. 
 Let's have a look at associated filed related to windDirection. 

204018 - Add associated field <br>
031021 - Associated field significance <br> 
011001 - windDirection <br>

Element **031021 - Associated field significance** is defined as a Code table. <br>
We decode its value with
>  **codes_get(bufr, 'windDirection->associatedField->associatedFieldSignificance')**

The value in our BUFR file is **5**. Now we need to look at the Code Table entries to undersand its meaning. 
> 	8-bit indicator of quality control 
(0 = Data checked and declared good; 1 = Data checked and declared suspect; 2 = Data checked and declared aggregated; 3 = Data checked and declared out of instrument range; 4 = Data checked, declared aggregated, and out of instrument range; 5 = Parameter is not measured at the station; 6 = Daily value not provided; 7 = Data unchecked, 8-254 = Reserved; 255 = Missing (QC info not available))

To obtain info of quality control indicator related to windDirection
>**codes_get(bufr, 'windDirection->associatedField)**

The value in our BUFR file is **0**, which correspond to 
> 0 = Data checked and declared good
